In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv, html
import lxml

# Regex
regex_html = r"(\<\/?[a-zA-Z0-9\=\"\_\s]+\>)"
regex_html_4_content = r"(\<\/?[a-zA-Z0-9\=\"\_\s\-]+\>)"
regex_d_space = r"(\s{2,})"
regex_weblink = r"(www.[A-Za-z\.]+)"

# File url
file_url = '../datasets/wsjhealth.txt'
dest_url = './run_data/wsj_scrapt_v2.csv'

# Basic list
csv_header = ['title','location','type_empl','wage','industries','content']

# Support scripts
# %run -i scripts.py

In [6]:
# Merge list of string into a long text.
def join_txt(list_txt):
    # Join list of content into 1 simple paragraph
    seperator = '. '
    return seperator.join(list_txt)

# Clear any html tags and decode the text
def clear_html_and_unescape(sample):
    # Clear html
    result = re.sub(regex_html, '', sample[0])
    
    # Clear Unescape
    result = html.unescape(result)
    
    return result

# Clean Web content
def clean_web_scrap_content(content):
    
    content = re.sub(regex_html_4_content,' ',content)
    content = html.unescape(content)
    content = re.sub(regex_d_space, ' ',content)
    content = re.sub(regex_weblink,'website',content)
    content = content.replace(u'\xa0', u' ').lstrip().rstrip()
    
    return content

In [7]:
# Test Cell

url_w_salary = 'https://www.seek.com.au/job/38961481?type=standout&searchrequesttoken=500a5225-80f9-4fcf-88c4-4985e717adf2'
url_wo_salary = 'https://www.seek.com.au/job/38952364?type=standout&searchrequesttoken=bd45b375-6e0c-4572-852c-1bfa7acc2952'

url_generous_salary = 'https://www.seek.com.au/job/38931990?type=standard&searchrequesttoken=ddc9ad23-06e8-42c5-9691-a35a9f7c6c5e'
url_test_random = 'https://www.seek.com.au/job/38843200?type=standard&searchrequesttoken=480f435f-3db5-4612-a554-a31df99c685e'

# XPath Strings
res = requests.get(url_test_random)
if res.status_code  == 200:

    print("Header: ", res.status_code)
    # Initialize XPath
    xpath_selector = Selector(text=res.text)
    
    # Title
    # //h1[@class="jobtitle"]
    title = xpath_selector.xpath('//h1[@class="jobtitle"]/text()').extract()

    # Location
    # //section[@role="region"]/dl/dd[2]/span/span
    location = xpath_selector.xpath('//section[@role="region"]/dl/dd[2]/span/span').extract()
    location = clear_html_and_unescape(location)
    
    # Type of Employee
    # //section[@role="region"]/dl/dd[3]/span/span
    type_employee = xpath_selector.xpath('//section[@role="region"]/dl/dd[3]/span/span').extract()
    type_employee = re.sub(regex_html,'', type_employee[0])
    
    # Wage
    # //*[contains(text(),'Salary')]
    is_Salary = xpath_selector.xpath('//*[contains(text(),"Salary")]/text()').extract()
    if is_Salary:
        # TODO
        wage = xpath_selector.xpath('//article/div[1]/div[2]/div/section/dl/div[1]/dd/span/span/text()').extract()
        wage = wage[0]
    else:
        wage = ''
        
    # Industries
    # //section[@role="region"]/dl/div/dd/span/span/strong
    industries = xpath_selector.xpath('//section[@role="region"]/dl/div/dd/span/span/strong').extract()
    industries = clear_html_and_unescape(industries)
    
    # Content
    # //div[@data-automation="mobileTemplate"]
    content = xpath_selector.xpath('//div[@data-automation="mobileTemplate"]').extract()
    content = join_txt(content)
    
    # Print-out
    print("Title: ", title[0])
    print("Location: ", location)
    print("Type Empl: ", type_employee)
    print("Wage: ", wage)
    print("Industries: ", industries)
    print("Content: ", len(content))
else:
    print("[ERROR]: ", res.status_code)

Header:  200
Title:  Store Manager | Hurstville | BIG W 
Location:  Sydney, South West & M5 Corridor
Type Empl:  Full Time
Wage:  
Industries:  Retail & Consumer Products
Content:  3581


In [8]:
content

'<div data-automation="mobileTemplate" class="_2e4Pi2B"><p> </p><div> <div> <div></div> </div> </div> <ul><li>Are you ready to make a BIG difference for families?</li> <li>Are you an inspiring leader who puts your team and our customers first in everything you do?</li> <li>Large trading store with lots of growth</li> </ul><p></p><p><strong>What we\'re like</strong></p> <p> </p><div>BIG W is one of Australia’s most loved brands and has been part of our families for over 40 years. At BIG W, we put our customer at the heart of everything we do across our network of 183 stores nationally. Our customers can choose everything for their kids, home, themselves, for leisure and for their everyday life, with exclusive BIG W brands that are simply great quality at low prices as well as feature leading international brands.</div> <div></div> <div>BIG W\'s best and most valued asset is its team of 18,000 passionate people located in Australia, Hong Kong and Bangladesh. Our core values of caring dee

In [9]:
content = clean_web_scrap_content(content)
content

"Are you ready to make a BIG difference for families? Are you an inspiring leader who puts your team and our customers first in everything you do? Large trading store with lots of growth What we're like BIG W is one of Australia’s most loved brands and has been part of our families for over 40 years. At BIG W, we put our customer at the heart of everything we do across our network of 183 stores nationally. Our customers can choose everything for their kids, home, themselves, for leisure and for their everyday life, with exclusive BIG W brands that are simply great quality at low prices as well as feature leading international brands. BIG W's best and most valued asset is its team of 18,000 passionate people located in Australia, Hong Kong and Bangladesh. Our core values of caring deeply, listening and learning and always doing the right thing are how we put team and customer first every day. The Role It takes a special kind of individual to run a BIG W store. You are the heart & mind o